## Install app requirements

In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt
!source .venv/bin/activate

___

### Load environment variables

In [1]:
from commons.constants import *
mdb_url = MONGO_PRODUCTS_COLLECTION_URL

## Connect to Spark Cluster using SparkConnect

In [2]:
from analytics.sc_session import get_session
session = get_session()
session

In [3]:
from analytics.sc_data import load_products_collection
df = load_products_collection(session)
df

DataFrame[_id: struct<product_id:bigint>, product_created_at: string, product_deleted_at: void, product_name: string, product_properties: string, product_sku: string, product_updated_at: string]

In [ ]:
df.printSchema()

In [ ]:
df.createOrReplaceTempView("mbdtfm_magento_products_tmp")
df5 = session.sql("""
    select _id.product_id, product_name
    from mbdtfm_magento_products_tmp
    where product_deleted_at is null
    order by product_name asc limit 5
  """)
df5.show()

In [4]:
df.show(n=10)

+----+--------------------+------------------+--------------------+--------------------+-----------+--------------------+
| _id|  product_created_at|product_deleted_at|        product_name|  product_properties|product_sku|  product_updated_at|
+----+--------------------+------------------+--------------------+--------------------+-----------+--------------------+
| {2}|2025-08-08T09:29:58Z|              NULL|Strive Shoulder Pack|{"product_sku":"2...|    24-MB04|2025-08-08T09:29:58Z|
| {3}|2025-08-08T09:29:58Z|              NULL|Crown Summit Back...|{"product_sku": "...|    24-MB03|2025-08-08T09:29:58Z|
| {5}|2025-08-08T09:29:58Z|              NULL|Rival Field Messe...|{"product_sku":"2...|    24-MB06|2025-08-08T09:29:58Z|
| {6}|2025-08-08T09:29:58Z|              NULL|     Fusion Backpack|{"product_sku": "...|    24-MB02|2025-08-08T09:29:58Z|
| {8}|2025-08-08T09:29:59Z|              NULL|     Voyage Yoga Bag|{"product_sku": "...|    24-WB01|2025-08-08T09:29:59Z|
|{10}|2025-08-08T09:29:5

In [ ]:
df.describe().show()

In [ ]:
df.columns

In [5]:
import json
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql import DataFrame
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from typing import Dict

def create_product_embeddings(df: DataFrame, properties_column: str = "product_properties", embeddings_column: str = "product_embeddings") -> None:
    """
    Reads products from MongoDB, calculates embeddings for the `product_properties`
    field in collection `mbdtfm_magento_catalog_products_mview`, and writes back to MongoDB
    with the new `product_embeddings` column in the same collection.
    :param df: DataFrame containing the `product_properties` column to create embeddings from.
    :param properties_column: Name of the `product_properties` column to create embeddings from.
    :param embeddings_column: Name of the `product_embeddings` column to create embeddings to.
    :return: None.
    """
    flatten_udf = udf(__flatten_json, StringType())
    df_flat = df.withColumn(f"{properties_column}_flat", flatten_udf(col(properties_column)))
    source_column = f"{properties_column}_flat"
    df = __compute_embeddings(df_flat, embeddings_column, source_column)
    (df.write.format("mongodb")
        .option("spark.mongodb.write.connection.uri", MONGO_URL)
        .option("database", MONGO_DATABASE)
        .option("collection", MONGO_PRODUCTS_COLLECTION)
        .mode("append")
        .save())


def __compute_embeddings(df: DataFrame, target_column: str, source_column: str) -> DataFrame:
    tokenizer = Tokenizer(inputCol=source_column, outputCol="tokens")
    tokenized = tokenizer.transform(df)
    hashing_tf = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=128)
    featurized = hashing_tf.transform(tokenized)
    idf = IDF(inputCol="rawFeatures", outputCol=target_column)
    idf_model = idf.fit(featurized)
    return idf_model.transform(featurized)

def __flatten_json(json_str: str) -> str:
    try:
        flat = __flatten(json.loads(json_str))
        return ' '.join([f"{k}={v}" for k, v in flat.items()])
    except Exception:
        return ''

def __flatten(obj: Dict, prefix: str = '') -> Dict[str, str]:
    """
    Recursive helper to flatten JSON objects inside a UDF, preparing data for tokenization.
    :param obj: JSON object.
    :param prefix: prefix to prepend to all keys.
    :return: flattened dictionary.
    """
    items = []
    for k, v in obj.items():
        new_key = f"{prefix}{k}" if prefix == '' else f"{prefix}.{k}"
        if isinstance(v, dict):
            items.extend(__flatten(v, new_key).items()) # Recursive call.
        elif isinstance(v, list):
            val_str = ' '.join([str(item) for item in v])
            items.append((new_key, val_str))
        else:
            items.append((new_key, str(v)))
    return dict(items)

In [6]:
create_product_embeddings(df)

df.columns

+---+--------------------+------------------+--------------------+--------------------+-----------+--------------------+-----------------------+
|_id|  product_created_at|product_deleted_at|        product_name|  product_properties|product_sku|  product_updated_at|product_properties_flat|
+---+--------------------+------------------+--------------------+--------------------+-----------+--------------------+-----------------------+
|{2}|2025-08-08T09:29:58Z|              NULL|Strive Shoulder Pack|{"product_sku":"2...|    24-MB04|2025-08-08T09:29:58Z|   product_sku=24-MB...|
|{3}|2025-08-08T09:29:58Z|              NULL|Crown Summit Back...|{"product_sku": "...|    24-MB03|2025-08-08T09:29:58Z|   product_sku=24-MB...|
|{5}|2025-08-08T09:29:58Z|              NULL|Rival Field Messe...|{"product_sku":"2...|    24-MB06|2025-08-08T09:29:58Z|   product_sku=24-MB...|
|{6}|2025-08-08T09:29:58Z|              NULL|     Fusion Backpack|{"product_sku": "...|    24-MB02|2025-08-08T09:29:58Z|   product

SparkException: Writing job failed.

JVM stacktrace:
org.apache.spark.SparkException
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobFailedError(QueryExecutionErrors.scala:893)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:452)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:397)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.writeWithV2(WriteToDataSourceV2Exec.scala:237)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:360)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:358)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.run(WriteToDataSourceV2Exec.scala:237)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.ex